In [ ]:
%%bash
# Crear carpeta Mllib en hadoop

hdfs dfs -mkdir /FuentesMllib

In [ ]:
%%bash
# Subir archivos spam.txt y normal.txt a la carpeta FuentesMllib en hadoop

hdfs dfs -copyFromLocal spam.txt /FuentesMllib
hdfs dfs -copyFromLocal normal.txt /FuentesMllib

In [ ]:
sc

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import SVMWithSGD

In [ ]:
# Leer archivos 

spam = sc.textFile("/FuentesMllib/spam.txt")
normal = sc.textFile("/FuentesMllib/normal.txt")

In [ ]:
# Se crea una instancia HashingTF para asignar un numero hasta 10000 palabras diferentes 

tf = HashingTF(numFeatures = 10000)

In [ ]:
# Se dividen los archivos por palabras y a cada palabra  se le aplica la transformacion a numero

spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
normalFeatures = normal.map(lambda email: tf.transform(email.split(" ")))

In [ ]:
# En este caso se crean conjuntos de datos con una estructura LabeledPoint 
# Se asigna 1 a spam y 0 a normal

positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = normalFeatures.map(lambda features: LabeledPoint(0, features))

# Las etiquetas son:

## SPAM = 1
## NORMAL = 0

In [ ]:
#Se hace la Union de los LabeledPoint

trainingData = positiveExamples.union(negativeExamples)

In [ ]:
trainingData.cache()

In [ ]:
#Ejecuta algoritmo SVM (Support Vector Machine) con SGD () para hacer entrenamiento

model = SVMWithSGD.train(trainingData)

## Probamos para un ejemplo positivo de spam y luego uno normal

In [ ]:
#Aplicamos la misma transformación HashingTF para obtener los vectores y luego aplicamos el modelo

posTest = tf.transform("Felicidades has ganado solo da clic ...".split(" "))

In [ ]:
# Imprimimos el resultado para positivo en SPAM

print ("Prediction for positive test example: %g" % model.predict(posTest))

In [ ]:
# Imprimimos el resultado correo NORMAL
negTest = tf.transform("Hola Israel ya empece con Spark Machine Learning ...".split(" "))

print ("Prediction for negative test example: %g" % model.predict(negTest))